In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob 
import os
import warnings
import networkx as nx
import matplotlib.colors as mcolors
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy.stats import pearsonr
from sklearn.covariance import GraphicalLassoCV
from networkx.generators.community import gaussian_random_partition_graph
import scipy.stats as stats
from scipy.stats import ttest_ind
import nibabel as nib 
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
pd.options.display.max_rows = None
pd.options.display.max_columns = None
import glob as glob
import math
#set random generator outside loop 
#seed everytrhing once at begibning and dont touch it 
np.random.seed(3)

#### For tau graphs, just do these steps, but on Tau suvr, stratify by amyloid quartile 

- For stratifying by amyloid status,  Royse 2021 Validation of PET positivity thresholds in centiloids established that 21 (or 20) is about so use this

#### Data has been preprocessed into amyloid quartiles for both adni and A4

## I need to verify they used the same freesurfer atlases (for region specific analyses)


#### General steps 
#### 1. A. Read ADNI data quartile csvs 
- These data have regional tau SUVRs, centiloid values, visit dates, and ID 
- (103, 228) in low 
- (107, 228) in med low 
- (100, 228) in med high 
- (101, 228) in high 

In [2]:
adni_cent_low = pd.read_csv('./adni/adni_amy_tau_merged_cent_low_quartile.csv')
adni_cent_med_low = pd.read_csv('./adni/adni_amy_tau_merged_cent_med_low_quartile.csv')
adni_cent_med_high = pd.read_csv('./adni/adni_amy_tau_merged_cent_med_high_quartile.csv')
adni_cent_high = pd.read_csv('./adni/adni_amy_tau_merged_cent_high_quartile.csv')

In [3]:
adni_cent_low.head(5)

,RID,TRANSVERSETEMPORAL,SUPERIORTEMPORAL,INFERIORTEMPORAL,FUSIFORM,POSTERIORCINGULATE,INSULA,PERICALCARINE,PUTAMEN,PARAHIPPOCAMPAL,SUPERIORPARIETAL,LATERALOCCIPITAL,INFERIORPARIETAL,CAUDALANTERIORCINGULATE,POSTCENTRAL,CAUDATE,SUPRAMARGINAL,CENTILOIDS,PRECENTRAL,TEMPORALPOLE,PARACENTRAL,TRACER,ISTHMUSCINGULATE,ACCUMBENS_AREA,LINGUAL,PALLIDUM,AMYGDALA,ROSTRALANTERIORCINGULATE,BANKSSTS,CUNEUS,MIDDLETEMPORAL,SUPERIORFRONTAL,PRECUNEUS
0,120,1.209,1.575,1.733,1.494,1.564,1.314,1.500,1.372,1.518,1.838,1.845,1.820,1.611,1.675,1.338,1.731,32.0,1.543,1.757,1.584,FBP,1.513,0.962,1.407,1.335,1.236,1.274,1.539,1.773,1.795,1.651,1.576
1,677,1.326,1.600,2.022,1.655,1.705,1.525,1.450,2.214,1.373,2.144,2.097,2.159,1.681,2.124,1.911,2.113,45.0,2.000,1.588,2.010,FBP,1.401,1.431,1.517,2.639,1.433,1.562,1.586,1.806,1.913,1.841,1.668
2,896,1.128,1.587,1.880,1.526,1.499,1.557,1.337,1.469,1.545,1.767,1.747,1.737,1.326,1.574,1.297,1.573,35.0,1.522,2.282,1.549,FBP,1.460,1.568,1.415,1.408,2.108,1.134,1.471,1.690,1.781,1.587,1.478
3,1280,1.216,1.660,1.850,1.433,1.384,1.182,1.546,1.301,1.564,1.915,2.137,1.929,1.222,1.623,1.088,1.701,33.0,1.536,1.464,1.561,FBP,1.404,1.426,1.504,1.665,1.298,1.192,1.473,1.694,1.801,1.584,1.507
4,2245,1.049,1.395,1.729,1.433,1.284,1.301,1.295,1.423,1.420,1.559,1.724,1.602,1.280,1.477,1.276,1.466,31.0,1.317,1.690,1.422,FBP,1.466,1.378,1.369,1.703,1.694,1.151,1.451,1.389,1.652,1.400,1.418


#### 1. B. Read A4 data 
- These data have regional tau SUVRs, centiloid values, visit dates, and ID 
- (101, 277) in low 
- (86, 277) in med low 
- (94, 277) in med high
- (92, 277) in high

In [4]:
a4_cent_low = pd.read_csv('./a4/a4_amy_tau_merged_cent_low_quartile.csv')
a4_cent_med_low = pd.read_csv('./a4/a4_amy_tau_merged_cent_med_low_quartile.csv')
a4_cent_med_high = pd.read_csv('./a4/a4_amy_tau_merged_cent_med_high_quartile.csv')
a4_cent_high = pd.read_csv('./a4/a4_adni_amy_tau_merged_cent_high_quartile.csv')

In [5]:
a4_cent_low.head(5)

,RID,TRANSVERSETEMPORAL,SUPERIORTEMPORAL,INFERIORTEMPORAL,FUSIFORM,POSTERIORCINGULATE,INSULA,PERICALCARINE,PUTAMEN,PARAHIPPOCAMPAL,SUPERIORPARIETAL,LATERALOCCIPITAL,INFERIORPARIETAL,CAUDALANTERIORCINGULATE,POSTCENTRAL,CAUDATE,SUPRAMARGINAL,CENTILOIDS,PRECENTRAL,TEMPORALPOLE,PARACENTRAL,TRACER,ISTHMUSCINGULATE,ACCUMBENS_AREA,LINGUAL,PALLIDUM,AMYGDALA,ROSTRALANTERIORCINGULATE,BANKSSTS,CUNEUS,MIDDLETEMPORAL,SUPERIORFRONTAL,PRECUNEUS
0,B10709603,0.982,1.056,1.184,1.235,0.962,1.091,1.034,1.583,1.161,0.901,1.054,1.090,0.932,0.880,1.133,1.036,40.593,0.920,1.069,0.912,Florbetapir,1.055,1.395,1.037,1.719,1.276,1.030,1.184,1.019,1.118,0.898,1.040
1,B11089276,1.066,1.093,1.158,1.182,1.128,1.220,1.135,1.679,1.089,1.015,1.066,1.134,1.089,0.995,1.269,1.098,35.101,1.049,1.040,1.054,Florbetapir,1.097,1.426,1.119,1.741,1.238,1.159,1.268,1.128,1.161,1.049,1.160
2,B12755986,0.956,1.045,1.145,1.167,0.974,1.115,0.943,1.574,1.027,0.997,0.997,1.125,1.024,0.882,1.119,1.037,33.270,0.898,0.917,0.956,Florbetapir,1.041,1.452,0.943,1.800,1.236,1.098,1.145,0.962,1.122,0.908,1.069
3,B14928511,0.945,1.039,1.154,1.135,0.962,1.115,1.015,1.568,1.078,0.847,1.042,1.027,0.995,0.844,1.132,0.988,22.286,0.889,1.074,0.855,Florbetapir,1.080,1.418,1.019,1.651,1.224,1.099,1.162,0.988,1.123,0.843,1.007
4,B16052018,1.117,1.091,1.144,1.242,1.128,1.223,1.147,1.532,1.157,1.020,0.996,1.084,1.198,0.964,1.213,1.050,36.932,1.003,1.138,1.094,Florbetapir,1.107,1.316,1.112,1.626,1.255,1.194,1.228,1.119,1.110,1.116,1.165


In [7]:
#look for and load in the A4 dataset because I think it has more patients

- out of 793 patients 
- if cut off is 24 then 
- 339 amy pos
- 454 amy neg

- if cut off is 40 then 
- 256 pos : if I do quartiles then this is 64 patients in each quartile, could be kinda small, maybe use this as the replication dataset if possible 
- 526 neg 

#### 2. Run and train separate graphical models for amyloid+ and amyloid- populations

##### I. Prepare data steps:
- a. need drop all non brain region vol cols - keep only brain regions 
- also drop the left and right ones 

- b. find out if there are any regions which have 0 volume or rows where there is no entry 

- c. normalize the data 

- d. check that there are no null values 

##### II. Create bootstrap samples and run model for amy pos

- a. create boot strap samples 

- b. run model 

- c. calc metrics 

##### III. Create bootstrap samples and run model for amy neg

- a. create boot strap samples 

- b. run model 

- c. calc metrics 

##### IV. Compare graph metrics using t test 

##### Later can Use GMM (2 components) to stratify patients into amyloid pos and neg 
- example code in mri_pet/pet_predicition_mldataframe.csv
- do thresholding on normal cognition 